# Advent of code Day 9 - Rope Bridge

### rules 
* tail must add next to head every step that separates them
* special cases of note:
    * diagonal is considered touching, so a add by H which results in T not sharing the same row and column but only 1 away is fine
    * any separation of HT while not int he same row and col that is beyond 1 T adds diagonally to close hte gap
    * allow overlaps

## Part 1 - Number of unique points T has visited
- include start point
- play H adds one at a time
- adds are defined as 
    - l ( 0,-1)
    - r ( 0, 1)
    - u ( 1, 0)
    - d (-1, 0)

```
x   u2l u2  u2r x
ul2 .   .   .   ur2
l2  .   T   .   r2
dl2 .   .   .   dr2
x   d2l d   d2r x
```
- if H adds within the coords of T or its neighbors ( the dots or T), T does nothing
- if H adds the ring outside the neighbors T must act ( the letters )
- H cannot reach the xs if we move one spot at a time

- may be helpful to keep an absolute coord of T for recording (to a set), and a relative coord on H from T
    - start point can be arbitrarily set to start

In [57]:
def get_inputs(filename): 
    with open(filename) as file: 
        lines = file.read().strip()
    commands = [ l.split(' ') for l in lines.split('\n') ]
    return [ (c[0],int(c[1])) for c in commands ]

assert get_inputs('../data/Day09-pre.txt') == [('R', 4), ('U', 4), ('L', 3), ('D', 1), ('R', 4), ('D', 1), ('L', 5), ('R', 2)]

In [58]:
# points as (row, col)
# x   u2l u2  u2r x
# ul2 .   .   .   ur2
# l2  .   T   .   r2
# dl2 .   .   .   dr2
# x   d2l d   d2r x

def add(x,y): return (x[0] + y[0], x[1] + y[1])
start,l,r,u,d   = (0,0),( 0,-1), ( 0, 1), ( 1, 0), (-1, 0)
ul,ur,dl,dr     = add(u,l), add(u,r), add(d,l), add(d,r)
u2,d2,r2,l2     = add(u,u), add(d,d), add(r,r), add(l,l)
u2l,u2r,ul2,ur2 = add(u2,l),add(u2,r),add(u,l2),add(u,r2)
d2l,d2r,dl2,dr2 = add(d2,l),add(d2,r),add(d,l2),add(d,r2)

def move(h, m):
    if   m == 'U': return add(h,u)
    elif m == 'D': return add(h,d)
    elif m == 'L': return add(h,l)
    elif m == 'R': return add(h,r)
    raise Exception(f'unknown move {m}')

def react(h, t, history):
    t_orig = t
    if h == u2:    h = u; t = add(t,u)
    elif h == u2r: h = u; t = add(t,ur)
    elif h == ur2: h = r; t = add(t,ur)
    elif h == r2:  h = r; t = add(t,r)
    elif h == dr2: h = r; t = add(t,dr)
    elif h == d2r: h = d; t = add(t,dr)
    elif h == d2:  h = d; t = add(t,d)
    elif h == d2l: h = d; t = add(t,dl)
    elif h == dl2: h = l; t = add(t,dl)
    elif h == l2:  h = l; t = add(t,l)
    elif h == ul2: h = l; t = add(t,ul)
    elif h == u2l: h = u; t = add(t,ul)
    
    if t != t_orig: history.add(t)
    return (h,t,history)
 
def part1(inputs):
    h,t,history = start, start, {start}
    for direction, count in inputs:
        for a in range(0,count):
            h,t,history = react(move(h,direction),t,history)
    return len(history)
            

print(f'Part 1: { part1(get_inputs("../data/Day09-pre.txt")) }')



################################################################

### test: movements of head
for m,result in [('U',u),('D',d),('L',l),('R',r)]:
    assert move(start,m) == result

### test: tail must move movements
assert react(u2, start,set()) == (u, u,  {u})
assert react(u2r,start,set()) == (u, ur, {ur})
assert react(ur2,start,set()) == (r, ur, {ur})
assert react(r2, start,set()) == (r, r,  {r})
assert react(dr2,start,set()) == (r, dr, {dr})
assert react(d2r,start,set()) == (d, dr, {dr})
assert react(d2, start,set()) == (d, d,  {d})
assert react(d2l,start,set()) == (d, dl, {dl})
assert react(dl2,start,set()) == (l, dl, {dl})
assert react(l2, start,set()) == (l, l,  {l})
assert react(ul2,start,set()) == (l, ul, {ul})
assert react(u2l,start,set()) == (u, ul, {ul})

### test: tail does not move
for h in [start, u, l, r, d, ur, ul, dr, dl]:
    assert react(h, start, set())  == (h, start, set())






Part 1: 13
